In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


###### Customers Cleaning


In [2]:
# Load the dataset from a CSV file
data = pd.read_csv('dim_customers.csv')
data.head()

,Customer ID,Order City,Order Postal,Order State,Latitude,Longitude
0,18287,Rock Springs,82901-7321,WY,41.5869,-109.2048
1,18283,JACKSON,83001-9460,WY,43.4800,-110.7618
2,18282,Greybull,82426,WY,44.4892,-108.0562
3,18281,EVANSTON,82930-4706,WY,41.2680,-110.9633
4,18280,CHEYENNE,82001-7901,WY,41.1400,-104.8202


In [3]:
# Display the shape of the dataset "number of rows and columns" 
data.shape

(4372, 6)

In [4]:
# Check for missing values in each column
data.isnull().sum()

Customer ID      0
Order City       0
Order Postal     0
Order State      0
Latitude        90
Longitude       90
dtype: int64

In [5]:
# Remove rows where 'Latitude' and 'Longitude' have missing values
data = data.dropna(subset=['Latitude', 'Longitude']) 

In [6]:
# Check for missing values in each column after cleaning
data.isnull().sum()

Customer ID     0
Order City      0
Order Postal    0
Order State     0
Latitude        0
Longitude       0
dtype: int64

In [7]:
# Remove Order Postal 
data = data.drop(['Order Postal'], axis=1)

In [8]:
# Convert city names to title case & Remove spaces
data["Order City"] = data["Order City"].str.title().str.strip()

In [9]:
# Convert state names to lowercase & Remove spaces
data["Order State"] = data["Order State"].str.lower().str.strip()

In [10]:
# Check for duplicated values in each column
data.duplicated().sum()

0

In [11]:
# Display the data
data

,Customer ID,Order City,Order State,Latitude,Longitude
0,18287,Rock Springs,wy,41.5869,-109.2048
1,18283,Jackson,wy,43.4800,-110.7618
2,18282,Greybull,wy,44.4892,-108.0562
3,18281,Evanston,wy,41.2680,-110.9633
4,18280,Cheyenne,wy,41.1400,-104.8202
...,...,...,...,...,...
4366,12352,Santa Ana,ca,33.7495,-117.8732
4367,12350,Riverside,ca,33.9534,-117.3962
4368,12349,Rio Vista,ca,38.1558,-121.6913
4370,12347,Pleasant Hill,ca,37.9480,-122.0608


In [12]:
# Save the cleaned data to a new CSV file
data.to_csv('Customers.csv', index=False)


###### Fact Sales Cleaning


In [13]:
# Load the dataset from a CSV file
df = pd.read_csv('fact_sales.csv')
df

,Transaction Date,Customer ID,Description,Stock Code,Invoice No,Quantity,Sales,Unit Price
0,12/1/2020 8:26,17850.0,Dog and Puppy Pads,85123A,536365.0,1,15.30,15.30
1,12/1/2020 8:34,13047.0,Pet Odor Eliminator,84879,536367.0,6,60.84,10.14
2,12/1/2020 8:34,13047.0,Rechargeable Pet Nail Grinder,22960,536368.0,1,25.50,25.50
3,12/1/2020 9:00,13748.0,Litter Slide Multi-Cat Scented Clumping Clay C...,22086,536371.0,14,214.20,15.30
4,12/1/2020 9:02,17850.0,Dog and Puppy Pads,85123A,536373.0,1,15.30,15.30
...,...,...,...,...,...,...,...,...
25060,12/9/2021 12:19,17581.0,Templation Soft Cat Treats,23084,581579.0,4,42.96,10.74
25061,12/9/2021 12:20,12748.0,ProBiotic Supplements for Dogs,79321,581580.0,1,34.50,34.50
25062,12/9/2021 12:23,13777.0,Pet Grooming Brush,20725,581583.0,7,60.90,8.70
25063,12/9/2021 12:31,15804.0,Pet Odor Eliminator,84879,581585.0,3,30.42,10.14


In [14]:
# Check for invalid data
# Find rows where 'Quantity', 'Unit Price', or 'Sales' are less than or equal to 0
invalid_values = df[(df['Quantity'] <= 0) | (df['Unit Price'] <= 0) | (df['Sales'] <= 0)]
print(invalid_values)

      Transaction Date  Customer ID  \
54     12/1/2020 12:38      17897.0   
195    12/2/2020 17:27      15384.0   
448    12/6/2020 12:55      14487.0   
454    12/6/2020 13:17      16550.0   
491    12/6/2020 15:09      16861.0   
...                ...          ...   
24909  12/8/2021 11:42      16933.0   
24944  12/8/2021 14:08      12476.0   
25002  12/8/2021 18:48      13078.0   
25004  12/8/2021 19:20      13883.0   
25005  12/8/2021 19:28      17924.0   

                                             Description Stock Code  \
54                         Rechargeable Pet Nail Grinder      22960   
195              Sheba Perfect Portions Pat Wet Cat Food      22197   
448    Taste of the Wild High Prairie Grain-Free Dry ...      22423   
454              Sheba Perfect Portions Pat Wet Cat Food      22197   
491    Litter Slide Multi-Cat Scented Clumping Clay C...      22086   
...                                                  ...        ...   
24909                             

In [15]:
# Removes any negative signs "making all values positive" by absolute function
df[['Unit Price', 'Sales', 'Quantity']] = df[['Unit Price', 'Sales', 'Quantity']].abs()

In [16]:
# Keep only rows where 'Quantity', 'Sales', and 'Unit Price' are greater than 0
df = df[(df['Quantity'] > 0) & (df['Sales'] > 0) & (df['Unit Price'] > 0)]

In [17]:
# Check for missing values in each column
df.isnull().sum()

Transaction Date       0
Customer ID         4354
Description            0
Stock Code             0
Invoice No           164
Quantity               0
Sales                  0
Unit Price             0
dtype: int64

In [18]:
# Fill missing Customer_IDs with "Unknown"
df['Customer ID'].fillna('Unknown', inplace=True)

C:\Users\pc-touch2023\AppData\Local\Temp\ipykernel_23956\2433312274.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Customer ID'].fillna('Unknown', inplace=True)


In [19]:
# Check for duplicated values
duplicate_rows = df[df.duplicated(subset=['Invoice No', 'Description', 'Quantity', 'Unit Price'], keep=False)]
print(duplicate_rows)

       Transaction Date Customer ID  \
54      12/1/2020 12:38     17897.0   
491     12/6/2020 15:09     16861.0   
683     12/8/2020 13:33     14299.0   
1077    12/14/2020 9:54     13802.0   
1079   12/14/2020 10:33     15093.0   
...                 ...         ...   
23439  11/25/2021 15:18     13069.0   
23659  11/28/2021 15:43     12657.0   
24041    12/1/2021 8:44     13078.0   
24879   12/8/2021 10:06     16019.0   
24880   12/8/2021 10:14     12558.0   

                                             Description Stock Code  \
54                         Rechargeable Pet Nail Grinder      22960   
491    Litter Slide Multi-Cat Scented Clumping Clay C...      22086   
683    Taste of the Wild High Prairie Grain-Free Dry ...      22423   
1077   Taste of the Wild High Prairie Grain-Free Dry ...      22423   
1079   Taste of the Wild High Prairie Grain-Free Dry ...      22423   
...                                                  ...        ...   
23439  Memory Foam Pet Beds for Sm

In [20]:
# Delete the duplicated values 
df = df.drop_duplicates(subset=['Invoice No', 'Description', 'Quantity', 'Unit Price'], keep=False)

In [21]:
# Check for missing values in each column after delete duplicated values
df.isnull().sum()

Transaction Date     0
Customer ID          0
Description          0
Stock Code           0
Invoice No          51
Quantity             0
Sales                0
Unit Price           0
dtype: int64

In [22]:
# Remove rows where 'Invoice No' has missing values
df = df.dropna(subset=['Invoice No'])

In [23]:
# Check for missing values in each column after cleaning
df.isnull().sum()

Transaction Date    0
Customer ID         0
Description         0
Stock Code          0
Invoice No          0
Quantity            0
Sales               0
Unit Price          0
dtype: int64

In [24]:
# Display the data
df

,Transaction Date,Customer ID,Description,Stock Code,Invoice No,Quantity,Sales,Unit Price
0,12/1/2020 8:26,17850.0,Dog and Puppy Pads,85123A,536365.0,1,15.30,15.30
1,12/1/2020 8:34,13047.0,Pet Odor Eliminator,84879,536367.0,6,60.84,10.14
2,12/1/2020 8:34,13047.0,Rechargeable Pet Nail Grinder,22960,536368.0,1,25.50,25.50
3,12/1/2020 9:00,13748.0,Litter Slide Multi-Cat Scented Clumping Clay C...,22086,536371.0,14,214.20,15.30
4,12/1/2020 9:02,17850.0,Dog and Puppy Pads,85123A,536373.0,1,15.30,15.30
...,...,...,...,...,...,...,...,...
25060,12/9/2021 12:19,17581.0,Templation Soft Cat Treats,23084,581579.0,4,42.96,10.74
25061,12/9/2021 12:20,12748.0,ProBiotic Supplements for Dogs,79321,581580.0,1,34.50,34.50
25062,12/9/2021 12:23,13777.0,Pet Grooming Brush,20725,581583.0,7,60.90,8.70
25063,12/9/2021 12:31,15804.0,Pet Odor Eliminator,84879,581585.0,3,30.42,10.14


In [25]:
# Save the cleaned data to a new CSV file
df.to_csv('Fact Sales.csv', index=False)


###### state_region_mapping Cleaning


In [26]:
# Load the dataset from a CSV file
dff = pd.read_csv('state_region_mapping.csv')
dff

,Order State,State Abbreviations,Region,State
0,AE,AE,Other,Armed Forces Europe
1,ak,AK,Other,Alaska
2,AK,AK,Other,Alaska
3,AL,AL,East,Alabama
4,ALABAMA,AL,East,Alabama
...,...,...,...,...
187,Wi,WI,Central,Wisconsin
188,wi,WI,Central,Wisconsin
189,Wisconsin,WI,Central,Wisconsin
190,WV,WV,East,West Virginia


In [27]:
# Convert state to lowercase & Remove spaces
dff["Order State"] = dff["Order State"].str.lower().str.strip()

In [28]:
# Check for duplicated values in each column 
dff.duplicated().sum()

83

In [29]:
# Delete the duplicated values 
dff = dff.drop_duplicates()

In [30]:
# Check for missing values in each column 
dff.isnull().sum()

Order State            0
State Abbreviations    0
Region                 0
State                  0
dtype: int64

In [31]:
# Display the data
dff

,Order State,State Abbreviations,Region,State
0,ae,AE,Other,Armed Forces Europe
1,ak,AK,Other,Alaska
3,al,AL,East,Alabama
4,alabama,AL,East,Alabama
5,alaska,AK,Other,Alaska
...,...,...,...,...
185,west virginia,WV,East,West Virginia
186,wi,WI,Central,Wisconsin
189,wisconsin,WI,Central,Wisconsin
190,wv,WV,East,West Virginia


In [32]:
# Save the cleaned data to a new CSV file
dff.to_csv('State Region Mapping.csv', index=False)


###### Products Cleaning


In [33]:
# Load the dataset from a CSV file
dfff = pd.read_csv('dim_products.csv')
dfff

,Stock Code,Weight,Landed Cost,Shipping_Cost_1000_mile,Description,Category
0,22423,40.0,35.0,20.0,Taste of the Wild High Prairie Grain-Free Dry ...,Food
1,85123A,2.0,6.0,5.0,Dog and Puppy Pads,Disposables
2,47566,6.0,18.0,12.0,"Memory Foam Pet Beds for Small, Medium, and La...",Disposables
3,85099B,2.0,5.0,4.0,Earth Rated Dog Poop Bags,Disposables
4,23084,2.0,6.0,6.0,Templation Soft Cat Treats,Pet Food
5,22086,7.0,9.0,9.0,Litter Slide Multi-Cat Scented Clumping Clay C...,Disposables
6,84879,1.0,4.0,4.0,Pet Odor Eliminator,Cleanig Supplies
7,79321,1.0,22.0,7.0,ProBiotic Supplements for Dogs,Supplements
8,22502,1.0,36.0,7.0,Indoor Pet Camera,Electronics
9,22197,0.5,2.5,2.5,Sheba Perfect Portions Paté Wet Cat Food,Food


In [34]:
# Check for missing values in each column 
dfff.isnull().sum()

Stock Code                 0
Weight                     0
Landed Cost                0
Shipping_Cost_1000_mile    0
Description                0
Category                   0
dtype: int64

In [35]:
# Check for duplicated values in each column 
dfff.duplicated().sum()

0

In [36]:
# Calculate the first quartile (Q1) and third quartile (Q3)
Q1 = dfff['Weight'].quantile(0.25)
Q3 = dfff['Weight'].quantile(0.75)

# Interquartile Range
IQR = Q3 - Q1  

# Determine the lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify values that are considered outliers
outliers = dfff[(dfff['Weight'] < lower_bound) | (dfff['Weight'] > upper_bound)]

# Display the outliers
print(f"Outliers:\n{outliers}")

Outliers:
   Stock Code  Weight  Landed Cost  Shipping_Cost_1000_mile  \
0       22423    40.0         35.0                     20.0   
15      23284    15.0         23.0                     10.0   

                                          Description     Category  
0   Taste of the Wild High Prairie Grain-Free Dry ...         Food  
15               Purina Pro Plan Adult Sensitive Skin  Supplements  


In [37]:
# Keep only the values that are NOT outliers
dfff = dfff[(dfff['Weight'] >= lower_bound) & (dfff['Weight'] <= upper_bound)]

In [38]:
# Display the data
print(dfff)

   Stock Code  Weight  Landed Cost  Shipping_Cost_1000_mile  \
1      85123A     2.0          6.0                      5.0   
2       47566     6.0         18.0                     12.0   
3      85099B     2.0          5.0                      4.0   
4       23084     2.0          6.0                      6.0   
5       22086     7.0          9.0                      9.0   
6       84879     1.0          4.0                      4.0   
7       79321     1.0         22.0                      7.0   
8       22502     1.0         36.0                      7.0   
9       22197     0.5          2.5                      2.5   
10      23298    10.0         18.0                     10.0   
11      22386     2.0          8.0                      6.0   
12      23203     1.0          6.0                      5.0   
13      21137     5.0         12.0                      9.0   
14      22720     1.0         20.0                      5.0   
16      22960     1.0         15.0                     

In [39]:
# Save the cleaned data to a new CSV file
dfff.to_csv('Products.csv', index=False)

-------------------------------------------------